Libraries setup

In [3]:
%pip install -q pip install openai prettytable python-dotenv pdfplumber

Note: you may need to restart the kernel to use updated packages.


  Retrying (Retry(total=4, connect=None, read=None, redirect=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x045F13B0>, 'Connection to pypi.python.org timed out. (connect timeout=15)')': /simple/install/
  Could not find a version that satisfies the requirement install (from versions: )
No matching distribution found for install


Api setup OpenAi

In [2]:
import openai
with open ("api.txt") as f:
    openai.api_key = f.read().strip()

In [3]:
openai.models.list()

SyncPage[Model](data=[Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'), Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'), Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'), Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created=169412247

In [6]:
%%writefile input.txt
Show me the top 5 highest-paid employees. 
Who is the youngest employee in the Engineering department? 
Find all employees who were hired in the year 2019. 
Show the names of employees whose salary is above the average salary in their department. 
Display the names of employees whose first and last names start with the same letter. 
Find the employee with the highest salary in each department   

Writing input.txt


Table setup using sqlite3 and pdfplumber

In [1]:
import pdfplumber

def extract_table_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        last_page = pdf.pages[-1]  # the table is on the last page
        table = last_page.extract_table()
        return table

file_path = '2024-06-20-Jr-AI-task-SQL.pdf'
table_data = extract_table_from_pdf(file_path)
print(table_data)


[['EmployeeID', 'FirstName', 'LastName', 'Age', 'Department', 'Position', 'Salary', 'HireDate', 'ManagerID'], ['1', 'John', 'Smith', '28', 'Sales', 'Manager', '85000', '01/03/2015', '21'], ['2', 'Jane', 'Doe', '34', 'Engineering', 'Developer', '95000', '23/05/2016', '6'], ['3', 'Emily', 'Johnson', '29', 'HR', 'Generalist', '70000', '14/08/2017', '7'], ['4', 'Michael', 'Brown', '45', 'Marketing', 'Coordinator', '80000', '11/11/2014', '20'], ['5', 'Sarah', 'Williams', '31', 'Sales', 'Consultant', '85000', '25/07/2013', '1'], ['6', 'David', 'Jones', '38', 'Engineering', 'Manager', '95000', '17/09/2019', '1'], ['7', 'Laura', 'Garcia', '26', 'HR', 'Manager', '70000', '02/12/2018', '21'], ['8', 'James', 'Miller', '39', 'Marketing', 'SEO', '80000', '10/01/2020', '20'], ['9', 'Anna', 'Davis', '27', 'Sales', 'Associate', '85000', '01/03/2015', '1'], ['10', 'Robert', 'Rodriguez', '41', 'Engineering', 'QA', '95000', '23/05/2016', '6'], ['11', 'Linda', 'Martinez', '33', 'HR', 'Coordinator', '70000

In [2]:
import sqlite3

def create_and_populate_db(data):
    # Connect to SQLite database
    conn = sqlite3.connect('employees.db')
    cursor = conn.cursor()

    # Create table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        EmployeeID INTEGER PRIMARY KEY,
        FirstName TEXT,
        LastName TEXT,
        Age INTEGER,
        Department TEXT,
        Position TEXT,
        Salary INTEGER,
        HireDate TEXT,
        ManagerID INTEGER
    )
    ''')

    # Prepare data (skip the header row of the table)
    prepared_data = [tuple(row) for row in data[1:]]

    # Insert data into table
    cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', prepared_data)
    conn.commit()
    conn.close()

# Assuming the first row of table_data contains headers
create_and_populate_db(table_data)
# show table
conn = sqlite3.connect('employees.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM employees')
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'John', 'Smith', 28, 'Sales', 'Manager', 85000, '01/03/2015', 21)
(2, 'Jane', 'Doe', 34, 'Engineering', 'Developer', 95000, '23/05/2016', 6)
(3, 'Emily', 'Johnson', 29, 'HR', 'Generalist', 70000, '14/08/2017', 7)
(4, 'Michael', 'Brown', 45, 'Marketing', 'Coordinator', 80000, '11/11/2014', 20)
(5, 'Sarah', 'Williams', 31, 'Sales', 'Consultant', 85000, '25/07/2013', 1)
(6, 'David', 'Jones', 38, 'Engineering', 'Manager', 95000, '17/09/2019', 1)
(7, 'Laura', 'Garcia', 26, 'HR', 'Manager', 70000, '02/12/2018', 21)
(8, 'James', 'Miller', 39, 'Marketing', 'SEO', 80000, '10/01/2020', 20)
(9, 'Anna', 'Davis', 27, 'Sales', 'Associate', 85000, '01/03/2015', 1)
(10, 'Robert', 'Rodriguez', 41, 'Engineering', 'QA', 95000, '23/05/2016', 6)
(11, 'Linda', 'Martinez', 33, 'HR', 'Coordinator', 70000, '14/08/2017', 7)
(12, 'William', 'Hernandez', 30, 'Marketing', 'Analyst', 80000, '11/11/2014', 20)
(13, 'Elizabeth', 'Lopez', 36, 'Sales', 'Analyst', 85000, '25/07/2013', 1)
(14, 'Richard', 'Gonzalez', 4

In [10]:
from openai import OpenAI

with open("api.txt") as f:
    api_key = f.read().strip()
client = OpenAI(api_key=api_key)
    
def read_questions(file_path):
    with open(file_path, 'r') as file:
        questions = file.readlines()
    return questions

def generate_sql(question):
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",  
        prompt="Given the following SQL tables, your job is to write queries given a user’s request.\n    \n    CREATE TABLE IF NOT EXISTS employees (\r\n        EmployeeID INTEGER PRIMARY KEY,\r\n        FirstName TEXT,\r\n        LastName TEXT,\r\n        Age INTEGER,\r\n        Department TEXT,\r\n        Position TEXT,\r\n        Salary INTEGER,\r\n        HireDate TEXT,\r\n        ManagerID INTEGER\r\n    )\n\n" + question,
        temperature=0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    sql_query = response.choices[0].text.strip()
    return sql_query

def execute_sql(sql_query):
    cursor.execute(sql_query)
    result = cursor.fetchall()
    return result

def optimize_sql(sql_query):
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"Optimize this SQL query: {sql_query}",
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].text.strip()

def main():
    questions = read_questions("input.txt")  # Path to your input file
    for question in questions:
        sql_query = generate_sql(question)
        optimization = optimize_sql(sql_query)
        query_result = execute_sql(sql_query)
        print("\n====\n Question:", question)
        print("Generated SQL Query:", sql_query)
        print("Query Results:", query_result)
        print("Optimization Suggestion:", optimization)

if __name__ == "__main__":
    main()
    conn.close()


Generated SQL Query: SELECT FirstName, LastName, Salary 
FROM employees 
ORDER BY Salary DESC 
LIMIT 5;
Query Results: [('Alex', 'Johnson', 200000), ('Jane', 'Doe', 95000), ('David', 'Jones', 95000), ('Robert', 'Rodriguez', 95000), ('Richard', 'Gonzalez', 95000)]
Optimization Suggestion: There is not much room for optimization in this query as it is already selecting only the necessary columns and limiting the number of results. However, here are a few suggestions that could potentially improve performance:

1. Add an index on the Salary column: This can improve the sorting process, especially if the table is large and the Salary column is frequently used for sorting.

2. Use a WHERE clause: If the query needs to retrieve data only for a specific group of employees (e.g. based on job title or department), adding a WHERE clause can help reduce the number of rows that need to be sorted.

3. Use a FETCH clause instead of LIMIT: If using a database system that supports the FETCH clause, it